# Distributed Operations

- Large-scale 모델은 크기가 크기 때문에 여러대의 GPU에 쪼개서 모델을 올려야 합니다.
- 그리고 쪼개진 각 모델의 조각들끼리 네트워크로 통신을 하면서 값을 주고 받아야 합니다.
- 이렇게 커다란 리소스를 여러대의 컴퓨터 혹은 여러대의 장비에 분산시켜서 처리하는 것을 '분산처리'라고 합니다.
- 이번 세션에서는 네트워크 통신에 사용되는 메시지 패싱의 개념과 분산프로그램 실행 방법, 주요 분산처리 연산 등에 대해 알아보겠습니다.

## 1. Message Passing

### Concept of Message Passing

- 메시지 패싱이란 동일한 주소공간을 공유하지 않는 여러 프로세스들이 데이터를 주고 받을 수 있도록 메시지라는 간접 정보를 주고 받는 것입니다. 
- 예를 들면 Process-1이 특정 태그가 달린 데이터를 메시지 큐에 send하도록, Process-2가 해당 데이터를 receive하도록 코딩해놓으면 두 프로세스가 공유하는 메모리 공간 없이도 데이터를 주고 받을 수 있죠.
- Large-scale 모델 개발시에 사용되는 분산 통신에는 대부분 이러한 message passing 기법이 사용됩니다.

![](../images/message_passing.png)

<br>

### MPI (Massage Passing Interface)
- MPI는 Message Passing에 대한 표준 인터페이스를 의미합니다.
- MPI에는 Process간의 Message Passing에 사용되는 여러 연산이 정의되어 있습니다. 
  - e.g. broadcast, reduce, scatter, gather, ...
- MPI의 대표적인 구현체로 OpenMPI라는 오픈소스가 존재합니다.

![](../images/open_mpi.png)

<br>

### NCCL & GLOO
- 실제로는 openmpi 보다는 nccl이나 gloo 같은 라이브러리를 사용하게 됩니다.
- NCCL (NVIDIA Collective Communication Library)
  - NVIDIA에서 개발한 GPU 특화 Message Passing 라이브러리 ('nickel'이라고 읽음)
  - NVIDIA GPU에서 사용시, 다른 도구에 비해 월등히 높은 성능을 보여주는 것으로 알려져있습니다.
- GLOO (Facebook's Collective Communication Library)
  - Facebook에서 개발된 Message Passing 라이브러리. 
  - `torch`에서는 주로 CPU 분산처리에 사용하라고 추천하고 있습니다.

### 백엔드 라이브러리 선택 가이드
- openmpi를 써야할 특별한 이유가 있는 것이 아니라면 nccl이나 gloo를 사용합니다.
- GPU에서 사용시 nccl, CPU에서 사용시 gloo를 사용하세요.
- 자세한건 https://pytorch.org/docs/stable/distributed.html 여기를 참고하세요.

<br>

### Multi-process 통신에 쓰이는 기본 용어
- Node: 일반적으로 컴퓨터라고 생각하시면 됩니다. 하나의 컴퓨터에는 여러 Device가 존재 할 수 있습니다.
- Global Rank: 원래는 프로세스의 우선순위를 의미하지만 **ML에서는 GPU의 ID**라고 보시면 됩니다.
- Local Rank: 원래는 한 노드내에서의 프로세스 우선순위를 의미하지만 **ML에서는 노드내의 GPU ID**라고 보시면 됩니다.
- World Size: 프로세스의 개수를 의미합니다.

![](../images/process_terms.png)


<br><br>

## 2. Multi-processing with PyTorch
- PyTorch로 구현된 Multi-processing 애플리케이션 튜토리얼을 진행합니다.

### Multi-process Application 실행 방법
- PyTorch로 구현된 Multi-process 애플리케이션을 실행시키는 방법은 크게 두가지가 있습니다.
- 이때, '분기한다.'라는 표현이 나오는데, 이는 한 프로세스가 부모가 되어 여러개의 서브프로세스를 동시에 실행시키는 것을 의미합니다.

### 1) 사용자의 코드가 부모 프로세스가 되어 특정 함수를 서브프로세스로 분기한다.

![](../images/multi_process_1.png)

<br>

- 이 경우는 사용자의 코드가 부모 프로세스가 되며 특정 function을 서브프로세스로써 분기합니다.
- 일반적으로 `Spawn`과 `Fork` 등 두가지 방식으로 서브 프로세스를 분기합니다.
- `Spawn`
  - 부모의 자원을 물려주지 않고 필요한 만큼의 자원만 서브프로세스에게 할당.
  - 속도가 느리지만 안전한 방식.
- `Fork`
  - 부모의 자원을 자식 프로세스와 공유하고 프로세스를 시작.
  - 속도가 빠르지만 위험한 방식.
- p.s. 실제로는 `Forkserver` 방식도 있지만 자주 사용되지 않는 생소한 방식이기에 생략합니다.


In [ ]:
"""
src/multi_process_1.py

참고:
Jupyter notebook은 멀티프로세싱 애플리케이션을 구동하는데에 많은 제약이 있습니다.
따라서 대부분의 경우 이곳에는 코드만 동봉하고 실행은 `src` 폴더에 있는 코드를 동작시키겠습니다.
실제 코드 동작은 `src` 폴더에 있는 코드를 실행시켜주세요.
"""

import torch.multiprocessing as mp


# 서브프로세스에서 동시에 실행되는 영역
def fn(rank, param1, param2):
    print(f"{param1} {param2} - rank: {rank}")


# 메인 프로세스
if __name__ == "__main__":
    processes = []
    # 시작 방법 설정
    mp.set_start_method("spawn")

    for rank in range(4):
        process = mp.Process(target=fn, args=(rank, "A0", "B1"))
        # 서브프로세스 생성
        process.daemon = False
        # 데몬 여부 (메인프로세스 종료시 함께 종료)
        process.start()
        # 서브프로세스 시작
        processes.append(process)

    for process in processes:
        process.join()
        # 서브 프로세스 join (=완료되면 종료)


In [14]:
!python ../src/multi_process_1.py

A0 B1 - rank: 0
A0 B1 - rank: 1
A0 B1 - rank: 2
A0 B1 - rank: 3


- `torch.multiprocessing.spawn` 함수를 이용하면 이 과정을 매우 쉽게 진행 할 수 있습니다.

In [ ]:
"""
src/multi_process_2.py
"""

import torch.multiprocessing as mp


# 서브프로세스에서 동시에 실행되는 영역
def fn(rank, param1, param2):
    # rank는 기본적으로 들어옴. param1, param2는 spawn시에 입력됨.
    print(f"{param1} {param2} - rank: {rank}")


# 메인 프로세스
if __name__ == "__main__":
    mp.spawn(
        fn=fn,
        args=("A0", "B1"),
        nprocs=4,  # 만들 프로세스 개수
        join=True,  # 프로세스 join 여부
        daemon=False,  # 데몬 여부
        start_method="spawn",  # 시작 방법 설정
    )


In [15]:
!python ../src/multi_process_2.py

A0 B1 - rank: 1
A0 B1 - rank: 2
A0 B1 - rank: 0
A0 B1 - rank: 3


In [ ]:
"""
참고: spawn.py

mp.spawn 함수는 아래와 같이 동작합니다.
"""

def start_processes(fn, args=(), nprocs=1, join=True, daemon=False, start_method='spawn'):
    _python_version_check()
    mp = multiprocessing.get_context(start_method)
    error_queues = []
    processes = []
    for i in range(nprocs):
        error_queue = mp.SimpleQueue()
        process = mp.Process(
            target=_wrap,
            args=(fn, i, args, error_queue),
            daemon=daemon,
        )
        process.start()
        error_queues.append(error_queue)
        processes.append(process)

    context = ProcessContext(processes, error_queues)
    if not join:
        return context

    # Loop on join until it returns True or raises an exception.
    while not context.join():
        pass


### 2) PyTorch 런처가 부모 프로세스가 되어 사용자 코드 전체를 서브프로세스로 분기한다.

![](../images/multi_process_2.png)

<br>

- `python -m torch.distributed.launch --nproc_per_node=n OOO.py`와 같은 명령어를 사용합니다.
- 이 방식은 torch에 내장된 멀티프로세싱 런처가 사용자 코드 전체를 서브프로세스로 실행시켜줘서 매우 편리합니다.

In [ ]:
"""
src/multi_process_3.py
"""

# 코드 전체가 서브프로세스가 됩니다.
import os

# RANK, LOCAL_RANK, WORLD_SIZE 등의 변수가 자동으로 설정됩니다.
print(f"hello world, {os.environ['RANK']}")

In [17]:
!python -m torch.distributed.launch --nproc_per_node=4 ../src/multi_process_3.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
hello world, 0
hello world, 1
hello world, 2
hello world, 3


### Process Group
- 많은 프로세스를 관리하는 것은 어려운 일입니다. 따라서 프로세스 그룹을 만들어서 관리를 용이하게 합니다.
- 프로세스 그룹을 초기화하는 `init_process_group` 함수는 반드시 서브프로세스에서 실행되어야 합니다.

In [ ]:
"""
src/process_group_1.py
"""

import torch.distributed as dist
# 일반적으로 dist와 같은 이름을 사용합니다.

dist.init_process_group(backend="nccl", rank=0, world_size=1)
# 프로세스 그룹 initialization
# 본 예제에서는 가장 자주 사용하는 nccl을 기반으로 진행하겠습니다.
# backend에 'nccl' 대신 'mpi'나 'gloo'를 넣어도 됩니다.

process_group = dist.new_group([0])
# 0번 프로세스가 속한 프로세스 그룹 생성

print(process_group)

In [18]:
!python ../src/process_group_1.py

Traceback (most recent call last):
  File "../src/process_group_1.py", line 8, in <module>
    dist.init_process_group(backend="nccl", rank=0, world_size=1)
  File "/home/ubuntu/kevin/kevin_env/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py", line 436, in init_process_group
    store, rank, world_size = next(rendezvous_iterator)
  File "/home/ubuntu/kevin/kevin_env/lib/python3.8/site-packages/torch/distributed/rendezvous.py", line 166, in _env_rendezvous_handler
    raise _env_error("MASTER_ADDR")
ValueError: Error initializing torch.distributed using env:// rendezvous: environment variable MASTER_ADDR expected, but not set


- 위 코드를 실행하면 에러가 발생합니다.
- 그 이유는 `MASTER_ADDR`, `MASTER_PORT` 등 필요한 변수가 설정되지 않았기 때문입니다.
- 이 값들을 설정하고 다시 실행시키겠습니다.

In [ ]:
"""
src/process_group_2.py
"""

import torch.distributed as dist
import os

# 일반적으로 이 값들도 환경변수로 등록하고 사용합니다.
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

# 통신에 필요한 주소를 설정합니다.
os.environ["MASTER_ADDR"] = "localhost"  # 통신할 주소 (보통 localhost를 씁니다.)
os.environ["MASTER_PORT"] = "29500"  # 통신할 포트 (임의의 값을 설정해도 좋습니다.)

dist.init_process_group(backend="nccl", rank=0, world_size=1)
# 프로세스 그룹 initialization

process_group = dist.new_group([0])
# 0번 프로세스가 속한 프로세스 그룹 생성

print(process_group)

In [19]:
!python ../src/process_group_2.py

- 위 예제는 프로세스 그룹의 API를 보여주기 위해서 메인프로세스에서 실행하였습니다. (프로세스가 1개 뿐이라 상관없었음)
- 실제로는 멀티프로세스 작업시 프로세스 그룹 생성 등의 작업은 반드시 서브프로세스에서 실행되어야 합니다.

In [ ]:
"""
src/process_group_3.py
"""

import torch.multiprocessing as mp
import torch.distributed as dist
import os


# 서브프로세스에서 동시에 실행되는 영역
def fn(rank, world_size):
    # rank는 기본적으로 들어옴. world_size는 입력됨.
    dist.init_process_group(backend="nccl", rank=rank, world_size=world_size)
    group = dist.new_group([_ for _ in range(world_size)])
    print(f"{group} - rank: {rank}")


# 메인 프로세스
if __name__ == "__main__":
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    os.environ["WORLD_SIZE"] = "4"

    mp.spawn(
        fn=fn,
        args=(4,),  # world_size 입력
        nprocs=4,  # 만들 프로세스 개수
        join=True,  # 프로세스 join 여부
        daemon=False,  # 데몬 여부
        start_method="spawn",  # 시작 방법 설정
    )

In [ ]:
!python ../src/process_group_3.py

## ?
- `gloo`, `nccl`, `openmpi` 등을 직접 사용해보는 것은 분명 좋은 경험이 될 것입니다. 그러나 시간 관계상 이들을 모두 다룰 수는 없고, 이들을 wrapping 하고 있는 `torch.distributed`와 이를 실행시킬 수 있는 `torch.multiprocess` 등의 패키지를  사용하여 진행하겠습니다.
- 실제로 활용 단으로 가면 `nccl` 등을 직접 사용하지 않고 대부분의 경우 `torch.distributed` 등의 하이레벨 패키지를 사용하여 프로그래밍 하게 됩니다.


## 2. P2P Communication
- P2P란 Peer to Peer의 준말로 한 프로세스와 다른 프로세스가 직접 통신하는 것을 의미합니다.
- 크게 위에서 설명한 `send`연산과 `recv`연산이 있으며, 이들을 비동기로 수행하려면 `isend`, `irecv`를 호출합니다.

In [ ]:
import torch
import torch.distributed as dist
from src

a_tensor = torch.tensor([])
